Note: Lab 5 now due Friday.
 - I refer to a `consleft` variable in the lab. That should be `conslost`.
 - Your NPV values should be on the order of -25, not -83.
 
Lab 6 due in 2 Fridays. Fairly short lab.

First: Nate's presentation

# Introduction

Generally, we have a lot of uncertainty about parameters (e.g., climate sensitivity), and we want to know how uncertainty in inputs propogates to uncertainty in results. Monte Carlo simulations are just many runs of a model with each parameters drawn from a distribution describing its uncertainty.

This is frequently used for studying climate hazards and impact analysis, and less frequently to understand uncertainty in mitigation (energy models).

As economists, we are also frequently interested in optimzation. This can be used for policy design (how should carbon prices be set?), as a mechanism to model human decision-making (how do people decide between adaptation and migration?), or to show other fields that they're doing things wrong (how suboptimal are current insurance programs?).

These naturally intersect for optimization under uncertainty, a classic but oddly under-studied problem in climate change.

# Monte Carlo simulations

It is related to bootstrap statistical tests and computational Bayesian model fitting.

We will use simple random sampling. More advanced treatment might use Latin Hypercube Sampling.

The basic setup is
$$y = f(\vec{x}, \vec{\beta})$$

In [ ]:
beta = rnorm(1000, 0, 1)

Very simple Monte Carlo simulation: let $$y = beta^2$$

In [ ]:
yy = beta^2

When there is an increase in downward radiation, surface temperatures change. This is expressed by
$$\Delta T = \lambda\, \Delta F$$
where $\Delta T$ is the change in surface temperature, $\Delta F$ is the change in downward radiation, called the radiative forcing, and $\lambda$ is some proportionality constant.

However, since a change in surface temperature will result in further radiative forcing. Let's model this:

Step 1. Initial change in forcing increases temperature:
$$\Delta T_0 = \lambda\, \Delta F_0$$

Step 2. Change in temperature causes additional forcing:
$$\Delta F_1 = C\, \Delta T_0$$

Step 3. Repeat steps 1 and 2 as follows:
$$\Delta T_i = \lambda\, \Delta F_i$$
$$\Delta F_{i+1} = C\, \Delta T_i$$
until $\Delta T_i$ goes to 0.

Now let's parameterize this:

"In the absence of feedback processes, climate models show λ ≡ λ0 = 0.30 to 0.31 [K/(W/m2)] (where λ0 is the reference climate sensitivity) (16), giving an equilibrium increase ΔT0 ≈ 1.2°C in response to sustained 2 × CO2." (Roe & Baker 2007)

"For the purposes of illustration, a normal distribution in hf(f) is shown with a mean of 0.65 and a SD of 0.13, typical to that obtained from feedback studies of GCMs (17, 18)." (Roe & Baker 2007)

In [ ]:
lambda = 0.30 # K/(W/m^2)
CC = 0.65 / lambda # this comes from f = lambda C
dF0 = 1.2 / lambda # W/m^2

In [ ]:
results = data.frame(tt=0, dF=dF0, dT=lambda * dF0)
for (tt in 1:100) {
    dF = CC * results$dT[nrow(results)]
    results = rbind(results, data.frame(tt, dF, dT=lambda * dF))
}

In [ ]:
results

In [ ]:
sum(results$dT)

In [ ]:
plot(cumsum(results$dT))

Now, let's do some Monte Carlo runs:

In [ ]:
results = data.frame()
for (ii in 1:1000) {
    ff = rnorm(1, .65, .13)
    CC = ff / lambda # this comes from f = lambda C

    dT = lambda * dF0
    DeltaT = dT
    for (tt in 1:100) {
        dF = CC * dT
        dT = lambda * dF
        DeltaT = DeltaT + dT
    }
    
    results = rbind(results, data.frame(ff, DeltaT))
}

In [ ]:
hist(results$ff)

# Optimization

The simple optimization problem is:
$$\text{argmax}_\gamma f(\vec{x}, \vec{\beta}, \gamma)$$

Things get complicated pretty quickly as you consider:
 - Multiple parameters
 - Functions with multiple local maxima

And there are many extensions: linear programming, multi-objective optimization, optimization-under-uncertainty, robust optimization. But that's mostly for another course. We will just use simple cases and ignore the issues.

R has two main optimization functions: `optim` and `optimize`.
 - `optimize` handles the simple one-dimensional case, where you can put bounds on the potential range of values.
 - `optim` is for the multi-dimensional case, and generally uses some kind of initial guess followed by gradient descent.

Very simple case: find the peak of a function.

In [ ]:
xx = seq(0, 6, by=.01)

In [ ]:
yy = dnorm(xx, 2)

In [ ]:
plot(xx, yy)

In [ ]:
optimize(function(guess) -dnorm(guess, 2), c(0, 6))

A more complicated case:

Simplest IAM I can construct.

In [ ]:
totalloss = function(mit) {
  co2cumul = cumsum(40 - mit) + 2500
  temp = co2cumul * 2 / 4000
  damages = 1.0038 * temp^2 / 100
  mitcosts = (100 * (mit^3/3) / 40^2) * 1e9 / (100*1e12)
  sum((mitcosts + damages) / (1 + 0.01^(1:length(mit))))
}

In [ ]:
soln = optim(rep(10, 2100 - 2020 + 1), totalloss, lower=0, upper=40)

In [ ]:
soln

In [ ]:
plot(2020:2100, soln$par)

# Optimization under uncertainty

## Simple case:

We are a agricultural insurer. The losses associated with a weather shock on a crop will be $\Delta T^2$ (so losses for increases or decreases that get worse for larger shocks). We don't know what $\Delta T$ will be, but we know that $\Delta T \sim N(0, 1)$.

We will have to pay for those losses. What should we set our premium at?

The analytical solution:

$$E[\Delta T^2] = Var(\Delta T) + E[\Delta T]^2 = 1$$

The wrong way:

Let's try run `optimize` containing a Monte Carlo to estimate the expected value!

In [ ]:
optimize(function(premium) {
    abs(premium - mean(rnorm(100, 0, 1)^2))
}, c(0, 6))

In this case, we could just calculate the mean of the distribution beforehand though...

In [ ]:
premium = mean(rnorm(100000, 0, 1)^2)

Make this a little more complicated. We want premium to affect the result...

In [ ]:
privatedraws = rnorm(1000, 0, sqrt(2)/2)
maxpremium = privatedraws^2 + .5
buyins = premium < maxpremium
sum((premium - (privatedraws + rnorm(1000, 0, sqrt(2)/2)))[buyins])

In [ ]:
optimize(function(premium) {
    privatedraws = rnorm(1000, 0, sqrt(2)/2)
    maxpremium = privatedraws^2 + .5
    buyins = premium < maxpremium
    sum((premium - (privatedraws + rnorm(1000, 0, sqrt(2)/2)))[buyins])
}, c(0, 6))

Solution: hold distributions constant

In [ ]:
privatedraws = rnorm(1000000, 0, sqrt(2)/2)
otherdraws = rnorm(1000000, 0, sqrt(2)/2)

In [ ]:
optimize(function(premium) {
    maxpremium = privatedraws^2 + .5
    buyins = premium < maxpremium
    sum((premium - (privatedraws + otherdraws))[buyins])
}, c(0, 6))